In [3]:
from selenium.webdriver.firefox.firefox_binary import FirefoxBinary
from selenium.webdriver.common.keys import Keys
from selenium import webdriver
from collections import defaultdict
import pandas as pd
from datetime import datetime, date
from lxml import etree as ET
from lxml.builder import E

# Contemporary Chosun Crawler
조선일보 2012, 2013, 2015, 2016, 2017년도 기사를 크롤링합니다.

In [5]:
# 여기 y_xxxx는 시작 날짜이고 시작 날짜부터 당해 12월 31일까지의 일수를 period에 입력하면 됩니다.

y_2012 = date.fromisoformat('2012-02-07')
y_2013 = date.fromisoformat('2013-01-22')
y_2015 = date.fromisoformat('2015-01-01')
y_2016 = date.fromisoformat('2016-01-19')
y_2017 = date.fromisoformat('2017-01-15')
datelist2012 = pd.date_range(y_2012, periods=329) # from 2012-01-14 to 2012-12-31
datelist2013 = pd.date_range(y_2013, periods=344) # from 2013-01-01 to 2013-12-31
datelist2015 = pd.date_range(y_2015, periods=365) # from 2015-01-01 to 2015-12-31
datelist2016 = pd.date_range(y_2016, periods=346) # from 2016-01-01 to 2016-12-31 (leap year)
datelist2017 = pd.date_range(y_2017, periods=351) # from 2017-01-01 to 2017-12-31

In [6]:
# 크롤링하고 싶은 연도를 입력하면 됩니다.
datelist = [] # 2012
for date in datelist2012: # 여기 연도를 바꾸세요!
    d = str(date).split(" ")[0]
    d = d.replace("-", "")
    datelist.append(d)
print(len(datelist)) # 1827 days to crawl

329


In [7]:
url_sample = "http://news.chosun.com/svc/list_in/list.html?indate="

urls=[]

for date in datelist:
    urls.append(url_sample+date)

In [8]:
class Article:
    title=""
    category=""
    paragraph=""

In [9]:
def text_writer(article, date):
    name = date + "_" + article.category + "_" + article.title
    txt = ""
    txt = txt + article.paragraph
    name = name.replace("/", " ")
    
    with open("./output/"+ name +".txt", "w") as text_file:
        text_file.write(txt)

def xml_writer(article, date):
    name = date + "_" + article.category + "_" + article.title
    name = name.replace("/", " ")
    title = E("title", article.title)
    category = E("category", article.category)
    paragraph = E("paragraph", article.paragraph)
    article_xml = E("article", category, title, paragraph)
    
    declaration = """<?xml version="1.0" encoding="utf-8"?>
    <!DOCTYPE articles SYSTEM "articles.dtd">
    """
    dec_byte = str.encode(declaration)
    with open("./output/"+ name +".xml", "wb") as f:
        f.write(dec_byte)
        f.write(ET.tostring(article_xml, xml_declaration=False, encoding="utf-8", pretty_print=True))
        
def article_extractor(article_urls, article_list, date):
    for article in article_urls:
        a = Article()
        browser.get(article)
        a.title = browser.find_elements_by_id("news_title_text_id")[0].text
        a.category = browser.find_elements_by_id("news_cat_trig_id")[0].text
        a.paragraph = browser.find_elements_by_class_name("par")[0].text
        text_writer(a, date)
        xml_writer(a, date)
        
        #article_list.append(a)
        print(date, len(article_list),"번째 기사: '", a.title, "'완료")

def list_extractor(page_url, article_list, date):
    articles = browser.find_elements_by_class_name("list_item")
    article_urls=[]
    for article in articles:
        if len(article.find_elements_by_class_name("author")) != 0:
            author = article.find_elements_by_class_name("author")[0]
            if len(author.text) >= 5:
                if author.text[:5] == "스포츠조선": # Excluding Sports Articles
                    pass
                else:
                    article_urls.append(article.find_elements_by_tag_name("a")[0].get_attribute("href"))
            else:
                article_urls.append(article.find_elements_by_tag_name("a")[0].get_attribute("href"))
    article_extractor(article_urls, article_list, date)

def page_browser(url, article_list): # browsing pages of that date
    i=1 # page index
    while len(browser.find_elements_by_class_name("list_item")) >= 1:
        if i == 1:
            page = "&pn=" + str(i)
            date = url[-8:]
            page_url = url + page
            browser.get(page_url)
        list_extractor(page_url, article_list, date)
        i += 1
        page = "&pn=" + str(i)
        page_url = url + page
        browser.get(page_url)
        
def contemporary_chosun_crawler(urls):
    article_list=[]
    for url in urls:
        browser.get(url)
        page_browser(url, article_list)

In [10]:
browser = webdriver.Firefox()
contemporary_chosun_crawler(urls)

20120207 0 번째 기사: ' [태평로] 부자 세금만으론 복지 흉내밖에 못 낸다 '완료
20120207 0 번째 기사: ' [우정아의 아트 스토리] [50] 잭슨 폴락, '넘버 1' '완료
20120207 0 번째 기사: ' [만물상] 한류 수출 '완료
20120207 0 번째 기사: ' [만물상] 한류 수출 '완료
20120207 0 번째 기사: ' [시론] 장시간 노동에 길든 한국인들 '완료
20120207 0 번째 기사: ' [특파원 칼럼] 롬니의 발목 잡는 富 '완료
20120207 0 번째 기사: ' [박두식 칼럼] 뒤집기의 達人들 '완료
20120207 0 번째 기사: ' [ESSAY] 내 친구의 詩 '완료
20120207 0 번째 기사: ' [편집자에게] 21세기의 새로운 외교 전쟁 '공공 외교' '완료
20120207 0 번째 기사: ' [편집자에게] 준비된 교사를 위한 인턴교사제 도입을 '완료
20120207 0 번째 기사: ' [편집자에게] 아이를 키우는 데 공짜는 없다 '완료
20120207 0 번째 기사: ' 가출 거부한 친구 담뱃불로 지진 여중생 '완료
20120207 0 번째 기사: ' 광주 장애인 복지시설 성폭행 의혹 '완료
20120207 0 번째 기사: ' 50대 경찰관의 자살 '완료
20120207 0 번째 기사: ' 영화 개봉실적에 따라 개발비 지원 '완료


NoSuchWindowException: Message: Browsing context has been discarded
